## Image Classification with Convolutional Neural Networks
We will revisit CNNs this time on CIFAR10, which has 60000 32x32 images from classes: aeroplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.losses import categorical_crossentropy
from keras import backend as K


A lot of the initialisation will be similar to what we have seen in the last practical. Lets start by defining the number of classes and some hyperparameters

In [16]:
num_classes = 10 #Cifar10 has 10 classes
batch_size = 128
epochs = 100

# input image dimensions
img_rows, img_cols = 32, 32

In [17]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape

(50000, 32, 32, 3)

As we did last time, lets normalise the images

In [18]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # This converts the pixel values from between 0 and 255 to between 0 and 1
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


As we did last practical, we want to set up y_train and y_test to have a vector of size 10 for each input sample.

In [19]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [20]:
y_train.shape # Make sure this is (50000,10)

(50000, 10)

### Building the model
We are going to build a model that combines everything we have looked at in the Lecture 5 - see the video "Putting it all together".


In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32,32,3), padding='same')) 
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten()) # This line is to convert from matrices to vectors
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax')) 
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization_16 (Batc (None, 32, 32, 16)        64        
_________________________________________________________________
dropout_20 (Dropout)         (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
batch_normalization_17 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
dropout_21 (Dropout)         (None, 16, 16, 32)       

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer='sgd',
              metrics=['accuracy'])

### Training the model
**Make sure you are using a GPU!** Otherwise this will be very slow.

It will take a little bit of time to train anyway so run the model, make a cup of tea, and watch some youtube (ideally my lectures!)

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/100
391/391 [==============================] - 3s 6ms/step - loss: 2.4442 - accuracy: 0.1923 - val_loss: 2.1184 - val_accuracy: 0.2246
Epoch 2/100
391/391 [==============================] - 2s 5ms/step - loss: 1.8142 - accuracy: 0.3305 - val_loss: 1.7173 - val_accuracy: 0.3602
Epoch 3/100
391/391 [==============================] - 2s 5ms/step - loss: 1.6706 - accuracy: 0.3827 - val_loss: 1.7714 - val_accuracy: 0.3645
Epoch 4/100
391/391 [==============================] - 2s 6ms/step - loss: 1.5846 - accuracy: 0.4148 - val_loss: 1.8023 - val_accuracy: 0.3600
Epoch 5/100
391/391 [==============================] - 2s 5ms/step - loss: 1.5320 - accuracy: 0.4365 - val_loss: 1.9201 - val_accuracy: 0.3585
Epoch 6/100
391/391 [==============================] - 2s 5ms/step - loss: 1.4735 - accuracy: 0.4535 - val_loss: 1.5857 - val_accuracy: 0.4234
Epoch 7/100
391/391 [==============================] - 2s 5ms/step - loss: 1.4385 - accuracy: 0.4701 - val_loss: 1.6796 - val_accuracy: 0.4315

The loss is still not as close to 0 as we would like, but we can see that it's still decreasing each epoch. This means that we should train it for longer to get maximum performance

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9833546876907349
Test accuracy: 0.6589999794960022


We get 66% performance which isn't great but also isn't too bad (random guessing would be 10%). Now lets trying building a deep model.

In [ ]:
deep_model = Sequential()
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32,32,3), padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu', padding='same', strides=2)) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3)) 
deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Flatten()) # This line is to convert from matrices to vectors
deep_model.add(Dense(128, activation='relu'))
deep_model.add(Dropout(0.3))
deep_model.add(Dense(10, activation='softmax')) 
deep_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_69 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_79 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_70 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_80 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 32, 32, 32)      

In [ ]:
deep_model.compile(loss=categorical_crossentropy,
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
deep_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = deep_model.evaluate(x_test, y_test, verbose=0)

Epoch 1/100
391/391 [==============================] - 15s 34ms/step - loss: 2.8149 - accuracy: 0.1256 - val_loss: 2.5942 - val_accuracy: 0.1865
Epoch 2/100
391/391 [==============================] - 13s 33ms/step - loss: 1.9413 - accuracy: 0.2800 - val_loss: 2.1596 - val_accuracy: 0.2665
Epoch 3/100
391/391 [==============================] - 13s 33ms/step - loss: 1.8015 - accuracy: 0.3278 - val_loss: 2.0895 - val_accuracy: 0.2827
Epoch 4/100
391/391 [==============================] - 13s 33ms/step - loss: 1.7138 - accuracy: 0.3616 - val_loss: 2.1830 - val_accuracy: 0.2851
Epoch 5/100
391/391 [==============================] - 13s 34ms/step - loss: 1.6489 - accuracy: 0.3878 - val_loss: 1.8909 - val_accuracy: 0.3450
Epoch 6/100
391/391 [==============================] - 13s 34ms/step - loss: 1.5934 - accuracy: 0.4136 - val_loss: 1.7971 - val_accuracy: 0.3784
Epoch 7/100
391/391 [==============================] - 13s 34ms/step - loss: 1.5322 - accuracy: 0.4362 - val_loss: 1.9142 - val_ac

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6878185272216797
Test accuracy: 0.7645000219345093


If you have the patience to stick this out to the end (or you are rich enough to own 1000 TPUs to train on), you can see we get better results already. 

If you skipped ahead, my results were 65.90% accuracy for the shallow model, and 76.45% accuracy for the deep model. Note that neither model had fully converged to a loss of zero.

### Exercise 1
After the practical is over, try training both models for 5000 epochs (in the background while you are doing something else - don't sit and watch the model while it trains). Which model gets better performance? Have the models finished training? Do the models overfit?

## Transfer Learning
Lets finish this notebook off by exploring how to use other people's trained models and how to fine tune them on our data. 


In [9]:
from keras.applications import ResNet50, resnet50
from keras.layers.experimental.preprocessing import Resizing

batch_size = 32

resnet = Sequential()   #We probably get better performance by resizing to 224,224 like the original resnet model, but that would be even slower. You can try this as an exercise
# resnet.add( Resizing(
#     224, 224, interpolation="bilinear", name=None
# ))
resnet.add( ResNet50(
    include_top=False,
    weights="imagenet", #this line allows us to load the pre-trained weights from ImageNet
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=None,
    classes=10))
    resnet.add(Flatten())
    resnet.add(Dense(10, activation='softmax')) # Add this line to get down to 10 output classes

94773248/94765736 [==============================] - 2s 0us/step


In [14]:
resnet.compile(loss=categorical_crossentropy,
              optimizer='adam', #Note that I have used adam here instead of sgd. It is a little bit more advanced.
              metrics=['accuracy'])

Finally, lets fit the model, stare at the screen, and pray that the loss goes down. Note this takes a really, really long time to train. You may want to go onto the next notebook and leave this one running in the background.

In [21]:
resnet.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = resnet.evaluate(x_test, y_test, verbose=0)

Epoch 1/100
391/391 [==============================] - 70s 167ms/step - loss: 1.2275 - accuracy: 0.5917 - val_loss: 22.3783 - val_accuracy: 0.1000
Epoch 2/100
391/391 [==============================] - 62s 160ms/step - loss: 0.9440 - accuracy: 0.6661 - val_loss: 10.4556 - val_accuracy: 0.1258
Epoch 3/100
391/391 [==============================] - 62s 159ms/step - loss: 0.8600 - accuracy: 0.6943 - val_loss: 9.4332 - val_accuracy: 0.1998
Epoch 4/100
391/391 [==============================] - 62s 159ms/step - loss: 0.7985 - accuracy: 0.7150 - val_loss: 6.5807 - val_accuracy: 0.2936
Epoch 5/100
391/391 [==============================] - 62s 159ms/step - loss: 0.7475 - accuracy: 0.7312 - val_loss: 9.3317 - val_accuracy: 0.2156
Epoch 6/100
391/391 [==============================] - 62s 159ms/step - loss: 0.7030 - accuracy: 0.7471 - val_loss: 3.8204 - val_accuracy: 0.4110
Epoch 7/100
391/391 [==============================] - 62s 158ms/step - loss: 0.6599 - accuracy: 0.7639 - val_loss: 6.8507

As before, because this takes some time to train, you may need to try these exercises after the practical has ended.

### Exercise 2
Copy and paste the above output into a Word Document.

Change resnet so it doesn't use pretrained weights from ImageNet - i.e. train it from scratch. How does your performance compare? Have a look at the train accuracy and val accuracy at each epoch, particularly in the first few epochs. What is happening here? Does it make sense?


### Exercise 3
The output of ResNet is a 2048 dimensional vector. This means we go from 2048 dimensions straight down to 10. We might be better off with an additional layer to decrease the dimensions a bit more gently. 

Add a new layer into the `resnet` model after Flatten that reduces the dimensionality to 256, before finally reducing the 10-dimensional output. 

(You can do this exercise with or without the ImageNet weights. Your choice

### Exercise 4 (Advanced) 
Start your model off with ImageNet weights and your 256-dimensional hidden layer. Freeze the ImageNet weights and train only the last two layers for the first five epochs.

### Exercise 5
See how ResNet50 performs on the cifar100 data set. Try with and without ImageNet weights.